In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
import os
import seaborn as sns
from tensorflow.python.framework import ops
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
%matplotlib inline

In [4]:
pwd

'C:\\Users\\sahil\\Projects\\Cars'

In [2]:
#dataset_dir = 'C:\\Users\\sahil\\Projects\\Cars\\fruits'

In [6]:
training_data = pd.DataFrame(columns=('name','path','label'))


os.chdir('C:\\Users\\sahil\\Projects\\Cars\\fruits')
all_subdirs = [name for name in os.listdir(".") if os.path.isdir(name)]



#for path, subdirs, files in os.walk('C:\\Users\\sahil\\Projects\\Cars\\car_data\\train'):   
for subs in all_subdirs:
    for path, subdirs, files in os.walk(subs):         
        for name in files:            
            training_data = training_data.append(pd.DataFrame({'name': name,'path': path, 'label': subs}, index=[0]), ignore_index=True)
        
        

In [7]:
training_data

,name,path,label
0,apple_01.jpg,apple,apple
1,apple_02.jpg,apple,apple
2,apple_03.jpg,apple,apple
3,apple_04.jpg,apple,apple
4,apple_05.jpg,apple,apple
...,...,...,...
6685,strawberry_974.jpg,strawberry,strawberry
6686,strawberry_975.jpg,strawberry,strawberry
6687,strawberry_976.jpg,strawberry,strawberry
6688,strawberry_98.jpg,strawberry,strawberry


In [8]:
TRAINING_DIR='C:\\Users\\sahil\\Projects\\Cars\\fruits'
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split= .2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale = 1./255,validation_split= .2,)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(64,64),
    color_mode='rgb',
    class_mode='categorical',
    subset='training',
    batch_size=32,
    #shuffle=TRUE,
    #seed=24
)



validation_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
 
    target_size=(64,64),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    subset='validation',
    #seed=24
)

Found 5353 images belonging to 7 classes.
Found 1335 images belonging to 7 classes.


In [9]:
model = tf.keras.models.Sequential([    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    # The third convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
  
    #  neuron hidden layer
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7, activation='softmax')
])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0

In [10]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
history = model.fit(train_generator, 
                    epochs=20, 
                    #steps_per_epoch=len(train_generator), 
                    validation_data = validation_generator, 
                    #verbose = 1, 
                    #validation_steps=len(validation_generator)
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 168 steps, validate for 42 steps
Epoch 1/20
168/168 [==============================] - 245s 1s/step - loss: 1.6559 - accuracy: 0.3574 - val_loss: 1.4841 - val_accuracy: 0.4607
Epoch 2/20
168/168 [==============================] - 35s 207ms/step - loss: 1.3865 - accuracy: 0.4730 - val_loss: 1.3340 - val_accuracy: 0.4959
Epoch 3/20
168/168 [==============================] - 39s 232ms/step - loss: 1.3138 - accuracy: 0.5078 - val_loss: 1.3803 - val_accuracy: 0.5011
Epoch 4/20
168/168 [==============================] - 38s 226ms/step - loss: 1.2455 - accuracy: 0.5358 - val_loss: 1.3221 - val_accuracy: 0.4914
Epoch 5/20
168/168 [==============================] - 38s 226ms/step - loss: 1.2129 - accuracy: 0.5556 - val_loss: 1.2563 - val_accuracy: 0.5236
Epoch 6/20
168/168 [==============================] - 38s 227ms/step - loss: 1.1642 - accuracy: 0.5739 - val_loss: 1.1941 - val_accuracy: 0.5356
Epoch 7/20
168/168 [===================